# Heatmaps computation from normalized LIME explanations

Aggregate local explanations (LIME) for every model in 'model_names' to compute global explanations in the form of heatmaps.

- Normalized explanations are loaded from subfolders under 'results_dir'.
- Heatmaps are stored in grayscale in 'heatmaps_dir_gray' and using the cv2.COLORMAP_JET colormap in 'heatmaps_dir_colormap'.

In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm

from ekman_expressions.heatmaps import save_heatmaps

In [17]:
# Models to evaluate
model_names = ['SilNet', 'WeiNet', 'AlexNet', 'SongNet', 'InceptionV3',
               'VGG19', 'VGG16', 'ResNet50', 'ResNet101V2', 'Xception',
               'MobileNetV3Large', 'EfficientNetV2B0']

# Number of k-cross validations and folder where they are located
# Alternatively set the paths to the target training and test manually
K = 5
dataset_paths_root = '../datasets/'
dataset_paths_train = []
dataset_paths_test = []
for i in range(K):
    dataset_paths_train.append(dataset_paths_root + 'CV' + str(i+1))
    dataset_paths_test.append(dataset_paths_root + 'CV' + str(i+1) + '_test')

# Folder where the evaluation results are saved
results_dir = '../results'

# Labels of the classes
label_names = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

# Dimensions of normalized images
norm_width = 224
norm_height = 275
vertical_space = 25

expl_method = 'lime'

# Folders where the heatmaps are saved
heatmaps_dir_colormap = '../heatmaps/COLORMAP_JET'
heatmaps_dir_gray = '../heatmaps/GRAY'

## Accumulate images and save grayscale and colormap heatmaps

In [ ]:
# Init progress bar
progress = tqdm(total=len(model_names)*len(dataset_paths_train)*len(label_names)*100)

# Init total accumulator
accum_total_img = np.zeros((norm_height, norm_width, len(label_names)), dtype=float)
accum_total_counter = np.zeros((len(label_names)), dtype=int)

# Create heatmaps folders if they don't exist
for f1 in [heatmaps_dir_colormap, heatmaps_dir_gray]:
    for f2 in ['heatmaps_by_cv', 'heatmaps_by_net', 'heatmaps_total']:
        if not os.path.exists(os.path.join(f1, f2)):
            os.mkdir(os.path.join(f1, f2))

# Iterate over each net
for model_name in model_names:

    # Init model accumulator
    accum_model_img = np.zeros((norm_height, norm_width, len(label_names)), dtype=float)
    accum_model_counter = np.zeros((len(label_names)), dtype=int)

    # Iterate over each CV set
    for train_path, test_path in zip(dataset_paths_train, dataset_paths_test):

        # Results foder
        results_net = os.path.join(results_dir, model_name + '_'
                                   + os.path.basename(train_path)
                                   + '_results')

        # 100 positives per class folder
        imgs100 = os.path.join(results_net, 'imgs_100')

        # Init CV set accumulator
        accum_cv_img = np.zeros((norm_height, norm_width, len(label_names)), dtype=float)
        accum_cv_counter = np.zeros((len(label_names)), dtype=int)

        # Iterate over each class
        for class_i, class_dir in enumerate(os.listdir(imgs100)):

            # Path of transformed LIME masks
            lime_masks_transformed_path = os.path.join(imgs100, class_dir, expl_method+'_masks_transformed')

            # Accumulate each mask
            for mask_name in os.listdir(lime_masks_transformed_path):

                # Load, to gray and to [0, 1] range
                mask = cv2.imread(os.path.join(lime_masks_transformed_path, mask_name))
                mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                mask = mask / 255

                # Accumulate mask
                accum_cv_img[:, : , class_i] += mask
                accum_cv_counter[class_i] += 1

                # Update progress
                progress.update(1)

            # Save heatmaps in grayscale, with threshold and in a colormap
            heatmap_name = model_name + '_' + os.path.basename(train_path) + '_' + label_names[class_i] + '_heatmap.png'
            save_heatmaps(
                img_accum = accum_cv_img[:, : , class_i] / accum_cv_counter[class_i],
                path_gray = os.path.join(heatmaps_dir_gray, 'heatmaps_by_cv', heatmap_name),
                path_colormap = os.path.join(heatmaps_dir_colormap, 'heatmaps_by_cv', heatmap_name),
                colormap = cv2.COLORMAP_JET
            )
            
        # Accumulate imgs and counters from a CV set
        accum_model_img += accum_cv_img
        accum_model_counter += accum_cv_counter

    # Save heatmap_model: with hist_stretch and color_map COLORMAP_JET
    for class_i in range(len(label_names)):
        heatmap_name = model_name + '_' + label_names[class_i] + '_heatmap.png'
        save_heatmaps(
            img_accum = accum_model_img[:, : , class_i] / accum_model_counter[class_i],
            path_gray = os.path.join(heatmaps_dir_gray, 'heatmaps_by_net', heatmap_name),
            path_colormap = os.path.join(heatmaps_dir_colormap, 'heatmaps_by_net', heatmap_name),
            colormap = cv2.COLORMAP_JET
        )

    # Accumulate imgs and counters from a CV set
    accum_total_img += accum_model_img
    accum_total_counter += accum_model_counter

# Save heatmap_model: with hist_stretch and color_map COLORMAP_JET
for class_i in range(len(label_names)):
    heatmap_name = label_names[class_i] + '_heatmap.png'
    save_heatmaps(
        img_accum = accum_total_img[:, : , class_i] / accum_total_counter[class_i],
        path_gray = os.path.join(heatmaps_dir_gray, 'heatmaps_total', heatmap_name),
        path_colormap = os.path.join(heatmaps_dir_colormap, 'heatmaps_total', heatmap_name),
        colormap = cv2.COLORMAP_JET
    )

# Close progress
progress.close()